# Notebook 3: The `scanpy` Workflow & Spatial QC with Visium

**Instructor:** Anthony Christidis 
**Time:** 45 minutes

---

Welcome to the computational analysis part of the workshop! Before we dive into our high-resolution Xenium data, let's learn the fundamental analysis workflow on a classic spatial technology: **10x Visium**.

In this notebook, we'll use `scanpy` for clustering and `squidpy` to demonstrate a best-practice Quality Control (QC) workflow that leverages spatial information. We will then apply a streamlined version of this workflow to our Xenium data to prepare it for the next notebook.

**Goals:**
1.  Perform a standard unsupervised clustering workflow on Visium data (`scanpy`).
2.  Visualize QC metrics (like total counts) in their physical space (`squidpy`).
3.  Visualize the final spot clusters on the tissue image.

### Setup and Data Loading

First, let's import our libraries and load the Visium Glioblastoma dataset.

In [ ]:
%load_ext jupyter_black

import spatialdata as sd
import scanpy as sc
import squidpy as sq

import warnings
warnings.filterwarnings("ignore")

sdata_visium = sd.read_zarr("../data/visium_glioblastoma_subset.zarr")
adata_visium = sdata_visium.tables["table"]

### Part 1: Visium Analysis - Spatial Quality Control

For spot-based data, visualizing QC metrics spatially is a critical first step. It can reveal technical issues like tissue detachment or slide artifacts.

First, we calculate the standard QC metrics using `scanpy`.

In [ ]:
sc.pp.calculate_qc_metrics(adata_visium, percent_top=(20, 50), inplace=True)

Now, we can use `squidpy`'s `spatial_scatter` function to plot these metrics. This function is perfectly optimized for Visium data, as it can automatically find the background image and scale factors from the `AnnData` object's metadata.

In [ ]:
import matplotlib.pyplot as plt

# We know these variables exist from your AnnData printout
adata_visium = sdata_visium.tables["table"]

# Get the spatial coordinates directly
coords = adata_visium.obsm['spatial']
x_coords = coords[:, 0]
y_coords = coords[:, 1]

# Create the two-panel plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Total counts
scatter1 = axes[0].scatter(x_coords, y_coords, c=adata_visium.obs['total_counts'], 
                          cmap='viridis', s=25, alpha=0.7)
axes[0].set_title('Total Counts per Spot')
axes[0].set_aspect('equal')
plt.colorbar(scatter1, ax=axes[0])

# Plot 2: Number of genes
scatter2 = axes[1].scatter(x_coords, y_coords, c=adata_visium.obs['n_genes_by_counts'], 
                          cmap='viridis', s=50, alpha=0.7)
axes[1].set_title('Unique Genes per Spot')
axes[1].set_aspect('equal')
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

These plots are essential. On the left, we can see areas with low `total_counts` which might correspond to regions with poor tissue quality. On the right, we see the gene complexity across the tissue.

### Part 2: Visium Analysis - `scanpy` Clustering

Based on our QC, let's filter the data and run the standard `scanpy` workflow to find transcriptionally distinct groups of spots, which should correspond to different tissue regions.

In [ ]:
print(f"Spots before filtering: {adata_visium.n_obs}")
sc.pp.filter_cells(adata_visium, min_counts=500)
sc.pp.filter_genes(adata_visium, min_cells=10)
print(f"Spots after filtering: {adata_visium.n_obs}")

In [ ]:
# Normalization and log-transformation
sc.pp.normalize_total(adata_visium, inplace=True)
sc.pp.log1p(adata_visium)

In [ ]:
# Find highly variable genes
sc.pp.highly_variable_genes(adata_visium)

In [ ]:
# Run PCA on the highly variable genes
sc.pp.pca(adata_visium, use_highly_variable=True)

In [ ]:
# Build neighborhood graph and run Leiden clustering
sc.pp.neighbors(adata_visium)
sc.tl.leiden(adata_visium, key_added="clusters")

In [ ]:
# Compute UMAP for visualization in abstract space
sc.tl.umap(adata_visium)

### Part 3: Visualizing the Visium Results

Let's visualize the clusters we found, both in UMAP space and back on the tissue.

In [ ]:
sc.pl.umap(adata_visium, color="clusters", title="Spot Clusters (UMAP)")

In [ ]:
# Get the current state of the data
adata_visium = sdata_visium.tables["table"]

# Check if clustering has been done
if 'clusters' not in adata_visium.obs.columns:
    print("Clusters not found. Running clustering workflow first...")
    
    # Quick clustering workflow
    sc.pp.normalize_total(adata_visium, inplace=True)
    sc.pp.log1p(adata_visium)
    sc.pp.highly_variable_genes(adata_visium)
    sc.pp.pca(adata_visium, use_highly_variable=True)
    sc.pp.neighbors(adata_visium)
    sc.tl.leiden(adata_visium, key_added="clusters")
    
    # Update the sdata object with our newly clustered data
    sdata_visium.tables["table"] = adata_visium
    
    print("Clustering complete!")
else:
    print("Using existing clusters.")

# Now we can safely plot the clusters
coords = adata_visium.obsm['spatial']
x_coords = coords[:, 0]
y_coords = coords[:, 1]

# Create the plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Create a scatter plot colored by clusters
scatter = ax.scatter(x_coords, y_coords, c=adata_visium.obs['clusters'].astype('category').cat.codes, 
                    cmap='tab10', s=25, alpha=0.8)
ax.set_title('Spot Clusters (Spatial View)')
ax.set_aspect('equal')

# Add a legend for the clusters
from matplotlib.lines import Line2D
import numpy as np

unique_clusters = adata_visium.obs['clusters'].unique()
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_clusters)))

legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[i], 
                         markersize=8, label=f'Cluster {cluster}') 
                  for i, cluster in enumerate(unique_clusters)]

ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

Excellent! We have successfully run a full analysis on Visium data and can see that the clusters `scanpy` found correspond to distinct histological regions.

### Part 4: Preparing the Xenium Data for the Next Notebook

Now that we've mastered the workflow, we will apply the same steps to our Xenium data in a single block. We will then save the processed object so we can dive straight into advanced spatial statistics in the next notebook.

In [ ]:
print("--- Processing Xenium Data ---")

# 1. Load Data
sdata_xenium = sd.read_zarr("../data/xenium_lung_cancer_subset.zarr")
adata_xenium = sdata_xenium.tables["table"].copy()

# 2. Filter
sc.pp.filter_genes(adata_xenium, min_cells=10)

# 3. Normalize and Log-transform
sc.pp.normalize_total(adata_xenium)
sc.pp.log1p(adata_xenium)

# 4. HVGs, PCA, Neighbors, Leiden, UMAP
sc.pp.highly_variable_genes(adata_xenium, n_top_genes=2000, flavor='seurat')
sc.pp.pca(adata_xenium, use_highly_variable=True)
sc.pp.neighbors(adata_xenium)
sc.tl.leiden(adata_xenium, key_added="clusters")
sc.tl.umap(adata_xenium)

print(f"Xenium data processed. Found {len(adata_xenium.obs['clusters'].unique())} clusters.")

In [ ]:
# 5. Save the processed AnnData object
import os
os.makedirs("../data/processed", exist_ok=True)

print("Saving processed Xenium AnnData object...")
adata_xenium.write("../data/processed/adata_xenium_processed.h5ad")
print("Done. We are ready for Notebook 4.")